In [1]:
import pandas as pd
import requests
import re
import cleaning_functions as cf

In [2]:
df_oscars = pd.read_csv('../data/raw/the_oscar_award.csv')
df_oscars.head(2)

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True


In [3]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10765 entries, 0 to 10764
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10765 non-null  int64 
 1   year_ceremony  10765 non-null  int64 
 2   ceremony       10765 non-null  int64 
 3   category       10765 non-null  object
 4   name           10761 non-null  object
 5   film           10450 non-null  object
 6   winner         10765 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 515.3+ KB


In [4]:
df_oscars.drop('ceremony',axis=1,inplace=True)

In [5]:
df_oscars.category = df_oscars.category.str.capitalize()

In [6]:
#quitar premios previos a 1967, ya que en este año se realizaron grandes cambios en la estructura de los premios (eliminación de categorías black and white)
df_oscars = df_oscars[df_oscars.year_film >= 1967]
df_oscars

,year_film,year_ceremony,category,name,film,winner
4451,1967,1968,Actor,Warren Beatty,Bonnie and Clyde,False
4452,1967,1968,Actor,Dustin Hoffman,The Graduate,False
4453,1967,1968,Actor,Paul Newman,Cool Hand Luke,False
4454,1967,1968,Actor,Rod Steiger,In the Heat of the Night,True
4455,1967,1968,Actor,Spencer Tracy,Guess Who's Coming to Dinner,False
...,...,...,...,...,...,...
10760,2022,2023,Honorary award,"To Euzhan Palcy, a masterful filmmaker who bro...",NaN,True
10761,2022,2023,Honorary award,"To Diane Warren, for her genius, generosity an...",NaN,True
10762,2022,2023,Honorary award,"To Peter Weir, a fearless and consummate filmm...",NaN,True
10763,2022,2023,Gordon e. sawyer award,Iain Neil,NaN,True


In [7]:
df_oscars.isnull().any()

year_film        False
year_ceremony    False
category         False
name              True
film              True
winner           False
dtype: bool

In [8]:
nan_rows_oscars = df_oscars[df_oscars.film.isnull()]

In [9]:
nan_rows_oscars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, 4557 to 10763
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      154 non-null    int64 
 1   year_ceremony  154 non-null    int64 
 2   category       154 non-null    object
 3   name           150 non-null    object
 4   film           0 non-null      object
 5   winner         154 non-null    bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 7.4+ KB


In [10]:
#comprobar contenido de las filas con nan values en películas. parece que es por premios especiales y premios honorarios, por lo que las eliminamos después
nan_rows_oscars.tail(5)

,year_film,year_ceremony,category,name,film,winner
10759,2022,2023,Jean hersholt humanitarian award,NaN,NaN,True
10760,2022,2023,Honorary award,"To Euzhan Palcy, a masterful filmmaker who bro...",NaN,True
10761,2022,2023,Honorary award,"To Diane Warren, for her genius, generosity an...",NaN,True
10762,2022,2023,Honorary award,"To Peter Weir, a fearless and consummate filmm...",NaN,True
10763,2022,2023,Gordon e. sawyer award,Iain Neil,NaN,True


In [11]:
df_oscars.dropna(subset='film', inplace = True)

In [12]:
df_oscars.isnull().any()

year_film        False
year_ceremony    False
category         False
name             False
film             False
winner           False
dtype: bool

In [13]:
subset = df_oscars[(df_oscars.year_film == 1967) & (df_oscars.winner == True)]

In [14]:
df_oscars = df_oscars.drop(df_oscars[df_oscars.category == 'Award of commendation'].index)

In [32]:
changes_category_values = {
    'Actress':'Actress in a leading role',
    'Actor': 'Actor in a leading role',
    'Art direction': 'Production design',
    'Best motion picture': 'Picture',
    'Directing':'Director',
    'Makeup': 'Makeup and hairstyling',
    'Sound':'Sound effects',
    'Sound effects editing':'Sound effects',
    'Sound editing': 'Sound effects',
    'Sound mixing':'Sound effects',
    'Foreign language film': 'International feature film'
}

In [33]:
changes_category_strings = {
    'subject': 'film',
    'cartoon': 'animated',
    '(':'',
    ')':'',
    'Special achievement award ':'',
    'Special ':'',  
    'dramatic ':''
}

In [16]:
df_oscars.category = cf.clean_strings_column(df_oscars.category, changes_category_strings)

In [17]:
df_oscars.category = df_oscars.category.str.capitalize()

In [18]:
df_oscars = cf.clean_values_column(df_oscars,changes_category_values)

In [19]:
#Crear sets con categorías similares para ver qué más se puede limpiar:

In [20]:
music_categories = []
for cat in df_oscars.category:
    if re.match(r'Music',cat,flags=re.IGNORECASE):
        music_categories.append(cat)
music_categories = set(music_categories)
music_categories

{'Music adaptation score',
 'Music original music score',
 'Music original musical or comedy score',
 'Music original score',
 'Music original score--for a motion picture [not a musical]',
 'Music original song',
 'Music original song score',
 'Music original song score and its adaptation -or- adaptation score',
 'Music original song score and its adaptation or adaptation score',
 'Music original song score or adaptation score',
 'Music score of a musical picture--original or adaptation',
 'Music scoring of music--adaptation or treatment',
 'Music scoring: adaptation and original song score',
 'Music scoring: original song score and adaptation -or- scoring: adaptation',
 'Music song',
 'Music song--original for the picture'}

In [21]:
music_patterns = {
    r'\b\w*scor\w*\b':'Original score'
}

In [22]:
music_clean = cf.value_replace_dictio_regex(music_categories, music_patterns, 'Original song')

In [24]:
df_oscars = cf.clean_values_column(df_oscars,music_clean)

In [31]:
writing_categories = []
for cat in df_oscars.category:
    if re.match(r'Writing',cat,flags=re.IGNORECASE):
        writing_categories.append(cat)
writing_categories = set(writing_categories)
writing_categories

In [26]:
writting_patterns = {
    r'\b\w*not preciously\w*\b':'Original screenplay',
    r'\b\w*directly\w*\b': 'Original screenplay',
    r'\b\w*original\w*\b': 'Original screenplay'
}

In [27]:
writing_clean = cf.value_replace_dictio_regex(writing_categories, writting_patterns, 'Adapted screenplay')

In [28]:
df_oscars = cf.clean_values_column(df_oscars,writing_clean)

In [29]:
df_oscars.nunique()

year_film          56
year_ceremony      56
category           24
name             4453
film             2927
winner              2
dtype: int64

In [30]:
df_oscars.category.unique()

array(['Actor in a leading role', 'Actor in a supporting role',
       'Actress in a leading role', 'Actress in a supporting role',
       'Production design', 'Cinematography', 'Costume design',
       'Director', 'Documentary feature', 'Documentary short film',
       'Film editing', 'International feature film', 'Original score',
       'Original song', 'Best picture', 'Short film animated',
       'Short film live action', 'Sound effects', 'Visual effects',
       'Adapted screenplay', 'Original screenplay',
       'Makeup and hairstyling', 'Animated feature film',
       'Documentary feature film'], dtype=object)

In [34]:
df_oscars.head(2)

,year_film,year_ceremony,category,name,film,winner
4451,1967,1968,Actor in a leading role,Warren Beatty,Bonnie and Clyde,False
4452,1967,1968,Actor in a leading role,Dustin Hoffman,The Graduate,False
